In [1]:
import deepdish as dd
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import deepdish as dd
import argparse
import numpy as np
import javabridge as jv
import bioformats as bf
from pyprind import prog_percent
from xml import etree as et
from queue import Queue
from threading import Thread, Lock
from enum import Enum
import torch as th
from torch.autograd import Variable
from torch.nn.functional import grid_sample
from tqdm import tqdm_notebook
import h5py
import os
from skimage.feature import blob_log
from skimage.draw import circle
from skimage.transform import AffineTransform, warp
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import matplotlib.cm as mplColorMap
import ipywidgets as widgets
#import ipyvolume as ipv
from scipy.interpolate import RegularGridInterpolator
import numpy as np
import deepdish as dd
from skimage.external.tifffile import imsave
from skimage.feature import register_translation
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.transform import AffineTransform, warp
from pyprind import prog_bar
import time

In [2]:
class Pyfish:

    def __init__(self, lif_file_path, save_path="", align_to_frame=0, use_gpu=True, max_displacement=300, thread_count=4):
        self.lif_file_path = lif_file_path
        self.align_to_frame = align_to_frame #un frami ke nesbat be un baghie axaro align mikonim
        self.use_gpu = use_gpu
        self.max_displacement = max_displacement
        self.thread_count = thread_count

        self._start_lif_reader()
        self.frame_shape=(1800, 1024, 1024)
        #self.frame_shape=


    # start lif reader
    # set image stack shape
    # lif files can contain multiple stacks so we pick the index of one with more than one frames in lif_stack_idx
    def _start_lif_reader(self):
        jv.start_vm(class_path=bf.JARS)

        log_level = 'ERROR'

        self.ir = bf.ImageReader(self.lif_file_path, perform_init=True)

        mdroot = et.ElementTree.fromstring(bf.get_omexml_metadata(self.lif_file_path))
        mds = list(map(lambda e: e.attrib, mdroot.iter('{http://www.openmicroscopy.org/Schemas/OME/2016-06}Pixels')))

        # lif can contain multiple images, select one that is likely to be the timeseries
        self.metadata = None
        self.lif_stack_idx = 0
        for idx, md in enumerate(mds):
            if int(md['SizeT']) > 1:
                self.lif_stack_idx = idx
                self.metadata      = md
        if not self.metadata: raise ValueError('lif does not contain an image with sizeT > 1')

    def read_frame(self, z):
        frame = np.empty(self.frame_shape, dtype=np.uint16)
        for t in range(1800):
            frame[t] = self._read_plane(t, z)
        return frame

    def _read_plane(self, t, z):
        return self.ir.read(t=t, z=z, c=0, series=self.lif_stack_idx, rescale=False)
    
    def andreas_registration(self):
        start=time.time()
        aligned = np.zeros((1800,21,1024,1024),dtype='float32')
        shifts = []
        
        for z in range(21):
            print('working on plane: ', z)
            dat=self.read_frame(z)
            #print(dat.shape)
            for t in range(1800):
                shift, err, phase = register_translation(dat[t], dat[0])
                at = AffineTransform(translation=shift)
                aligned[t, z] = warp(dat[t], at, preserve_range=True)

                shifts.append([z, t, np.sqrt(np.sum(shift**2)), err, phase])
        end=time.time()
        shifts=np.save(shifts)
        print('time taken:', end-start)
        dd.io.save('C:/Users/koester_lab/Documents/Maria/registered/fish17_6dpf_medium' + '_aligned_andreas.h5', aligned, None)
        np.save('C:/Users/koester_lab/Documents/Maria/registered/fish17_6dpf_medium' + '_shifts_andreas.npy', shifts)
                
                


In [ ]:
data_processor=Pyfish('//ZMN-HIVE/User-Data/Maria/control/fish17_6dpf_medium.lif')
data_processor.andreas_registration()

working on plane:  0
working on plane:  1
working on plane:  2
working on plane:  3
working on plane:  4
working on plane:  5
